# Testing S-57 Data Processing Classes

## 1. Setup

In [1]:
import sys
import os
from distutils.command.config import config
from pathlib import Path
from dotenv import load_dotenv
# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")

from src.maritime_module.core.s57_data import S57Base, S57Advanced, S57Updater, PostGISManager, SpatiaLiteManager, GPKGManager, S57AdvancedConfig
# Define paths for data and output
s57_data_dir = project_root / 'data' / 'ENC_ROOT'
s57_data_update_dir = project_root / 'data' / 'ENC_ROOT_UPDATE'
output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# Define database parameters from environment variables
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

print(f"S-57 data directory: {s57_data_dir}")
print(f"Output directory: {output_dir}")
print(f"PostGIS DB_name: {db_params['dbname']}")

/tmp/ipykernel_41268/1886726326.py:3: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.command.config import config


S-57 data directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/data/ENC_ROOT
Output directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output
PostGIS DB_name: ENC_db


In [ ]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
print(pg.get_schemas())

In [ ]:
from osgeo import gdal
print("GDAL Python bindings version:", gdal.__version__)
print("GDAL C library version:", gdal.VersionInfo("RELEASE_NAME"))

## 2. Test S57Base: Bulk Conversion

#### Convert to GPKG

In [ ]:
# import logging
# logging.getLogger('src.maritime_module.core.s57_data').setLevel(logging.DEBUG)

base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=str(output_dir / 'by_enc'),
        output_format='gpkg',
        overwrite=True
    )
base_converter.convert_by_enc()

#### Convert to PostGIS

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=db_params,
        output_format='postgis',
        overwrite=False
    )
base_converter.convert_by_enc()

In [ ]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
pg.get_schema_summary()

#### Convert to SpatiaLite

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
         output_dest=str(output_dir / 'by_enc'),
        output_format='spatialite',
        overwrite=True
    )
base_converter.convert_by_enc()

## 3. Test S57Advanced: Layer-centric Conversion

This section provides comprehensive tests for the `S57Advanced` class, which performs optimized, layer-centric conversions. We will test conversions to all supported formats (PostGIS, GeoPackage, SpatiaLite) and verify the results.

### 3.1. Convert to PostGIS and Verify

First, we convert the S-57 data into a layer-centric PostGIS database schema. Then, we use `PostGISManager` to verify the output.

In [ ]:
# --- Step 1: Conversion ---
pg_schema = 's57_advanced'
print(f"--- Starting S57Advanced conversion to PostGIS schema: '{pg_schema}' ---")

advanced_converter_pg = S57Advanced(
    input_path=s57_data_dir,
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=pg_schema,
    config=S57AdvancedConfig(auto_tune_batch_size=True, enable_debug_logging=False)
)
advanced_converter_pg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=pg_schema)

    # Verify a common layer exists and has data
    depare_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(depare_layer)} features from LNDMRK layer.")
    print("LNDMRK layer head (note the 'dsid_*' stamping columns):")
    display(depare_layer.head())

    # Verify feature stamping integrity across all layers
    print("\nVerifying feature update status against DSID table...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during PostGIS verification: {e}")


### 3.2. Convert to GeoPackage and Verify

Next, we test the conversion to a single GeoPackage file. The `GPKGManager` is used for verification.

In [ ]:
# --- Step 1: Conversion ---
gpkg_path = output_dir / 's57_advanced.gpkg'
print(f"--- Starting S57Advanced conversion to GeoPackage: '{gpkg_path}' ---")

advanced_converter_gpkg = S57Advanced(
    input_path=s57_data_dir,
    output_dest=str(gpkg_path),
    output_format='gpkg',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_gpkg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = GPKGManager(gpkg_path=gpkg_path)

    # Verify a common layer exists and has data
    soundg_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(soundg_layer)} features from LNDMRK layer in the GeoPackage.")
    print("LNDMRK layer head:")
    display(soundg_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during GeoPackage verification: {e}")


### 3.3. Convert to SpatiaLite and Verify

Finally, we test the conversion to a SpatiaLite database file and verify it with `SpatiaLiteManager`.

In [ ]:
# --- Step 1: Conversion ---
sqlite_path = output_dir / 's57_advanced.sqlite'
print(f"--- Starting S57Advanced conversion to SpatiaLite: '{sqlite_path}' ---")

advanced_converter_sqlite = S57Advanced(
    input_path=s57_data_dir,
    output_dest=str(sqlite_path),
    output_format='spatialite',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_sqlite.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = SpatiaLiteManager(db_path=sqlite_path)

    # Verify a common layer exists and has data
    boyspp_layer = manager.get_layer('boyspp')
    print(f"Loaded {len(boyspp_layer)} features from BOYSPP layer in the SpatiaLite DB.")
    print("BOYSPP layer head:")
    display(boyspp_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during SpatiaLite verification: {e}")


### 3.4. Test with Enterprise-Safe Parallel Processing

This test demonstrates using the `S57AdvancedConfig` to enable high-safety, read-only parallel processing, which can speed up the initial file discovery and preprocessing steps.

In [ ]:
# --- Step 1: Configure and Convert ---
parallel_schema = 's57_parallel_test'
print(f"--- Starting S57Advanced conversion with PARALLEL processing to schema: '{parallel_schema}' ---")

# Use the recommended high-safety configuration for parallel reads
high_safety_config = S57AdvancedConfig(
    enable_parallel_processing=True,
    parallel_read_only=True,
    parallel_db_writes=False,
    parallel_validation_level='strict',
    max_parallel_workers=2,
    enable_debug_logging=True
)

print("\nUsing configuration:")
print(high_safety_config.get_configuration_summary())

advanced_converter_parallel = S57Advanced(
    input_path=s57_data_dir,
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=parallel_schema,
    config=high_safety_config
)
advanced_converter_parallel.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=parallel_schema)
    summary = manager.get_enc_summary()
    print(f"Successfully created {len(summary)} ENCs in the parallel-processed schema.")
    display(summary.head())
except Exception as e:
    print(f"An error occurred during parallel processing verification: {e}")


## 4. Test S57Updater

The `S57Updater` is designed to perform incremental, transactional updates on a layer-centric database created by `S57Advanced`. We will use the schema created in test 3.1 (`s57_advanced_test`) as the target for our update.

**Note:** This test requires a running PostGIS database.

Update check:
Old Edition  -> New Edition |
US1EEZ1M 9:0  -> 9:0 |
US1GC09M 65:5 -> 71:1 |
US1PO02M 21:1 -> 21:1 |
US2WC12M 27:8 -> 27:12 |
US3CA52M 31:0 -> 31:7 |
US4CA60M 37:6 -> 38:2

### PostGIS Updater

In [ ]:
db_schema = 's57_advanced' # Use the schema created by S57Advanced
# Example S-57 file to update,
updater = S57Updater(output_format='postgis',
                     dest_conn=db_params,
                     schema=db_schema,
                     )
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()
display(summary_df)

### 4.2. SpatiaLite Updater

This test demonstrates the `S57Updater` functionality for a SpatiaLite database. The previous implementation failed with a `database disk image is malformed` error because it attempted to run the updater on a file that was being accessed by multiple, uncoordinated connections (OGR and SQLAlchemy).

The corrected test follows a robust, isolated process:

1.  **Create a fresh SpatiaLite database** from the initial ENC data (`s57_data_dir`). This provides a clean, known state to update and prevents file corruption.
2.  **Run the updater** on this new database using the update data (`s57_data_update_dir`).
3.  **Verify the results** by comparing the ENC versions before and after the update and checking for data integrity.

In [ ]:
sqlite_update_target_path = output_dir / 's57_advanced.sqlite'
# Example S-57 file to update,
updater = S57Updater(
     output_format='spatialite',
     dest_conn=str(sqlite_update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

### GPKG Updater

In [ ]:
update_target_path = output_dir / 's57_advanced.gpkg'
# Example S-57 file to update,
updater = S57Updater(
     output_format='gpkg',
     dest_conn=str(update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

## DeepTest

#### Setup

In [2]:
from tests.core__real_data.deep_test_s57_workflow import *

db_params_test = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db_schema = 's57_deeptest'
test_output_path = project_root / 'test_output'

test_config = TestConfig(s57_data_root = s57_data_dir,
                         s57_update_root=s57_data_update_dir,
                         test_output_dir = test_output_path,
                         test_level = 3,

                         skip_postgis = False,
                         skip_updates = True,
                         cleanup_on_success = True,
                         postgis_config = db_params_test,
                         test_schema_name = db_schema,
                         clean_output = False
)


print(f"\n--- Starting S57 Deep Test process with TEST CONFIG ---")
print(f"\n--- SettingUp directories:")
print(f"\nTest dataset at:       {test_config.s57_data_root}")
print(f"\nUpdate dataset at:     {test_config.s57_update_root}")
print(f"\nTest output directory: {test_config.test_output_dir}")
print(f"\n-----------------------------------")
post_conf = (f"Dababase: {db_params_test['dbname']} | Schema/Filename: {db_schema}") if  test_config.skip_postgis == False else ""
print(f"\nPostGIS tests:   {'❌' if test_config.skip_postgis == True else '✅'} {post_conf}")
print(f"\nUpdates process: {'❌' if test_config.skip_updates == True else '✅'}")



--- Starting S57 Deep Test process with TEST CONFIG ---

--- SettingUp directories:

Test dataset at:       /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/data/ENC_ROOT

Update dataset at:     /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/data/ENC_ROOT_UPDATE

Test output directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/tests/test_output

-----------------------------------

PostGIS tests:   ✅ Dababase: ENC_db | Schema/Filename: s57_deeptest

Updates process: ❌


#### Load Dataset

In [3]:
try:
    print(f"\n🎉 Commence DeepTest !")
    tester = S57DeepTester(test_config)
except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

# Testing Update Readiness
if test_config.skip_updates == False:
    compare_df = tester.analyze_update_readiness()
    display(compare_df)



🎉 Commence DeepTest !
2025-09-12 21:25:56,860 - tests.core__real_data.deep_test_s57_workflow - INFO - Setting up DeepTest environment...
2025-09-12 21:25:56,864 - tests.core__real_data.deep_test_s57_workflow - INFO - Found 6 S57 files for testing
2025-09-12 21:25:56,878 - tests.core__real_data.deep_test_s57_workflow - INFO - Cleaning existing outputs with same names...
2025-09-12 21:25:57,021 - tests.core__real_data.deep_test_s57_workflow - INFO - Dropping existing PostGIS schema: s57_deeptest
2025-09-12 21:25:57,022 - tests.core__real_data.deep_test_s57_workflow - WARNING - Could not clean PostGIS schema: This connection has already initialized a SQLAlchemy Transaction() object via begin() or autobegin; can't call begin() here unless rollback() or commit() is called first.
2025-09-12 21:25:57,033 - tests.core__real_data.deep_test_s57_workflow - INFO - PostGIS connectivity verified


In [4]:
try:
    report = tester.run_comprehensive_test()

    print(f"\n🎉 DeepTest completed successfully!")
    print(f"📁 Results saved to: {test_config.test_output_dir}")

except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

2025-09-12 21:25:59,945 - tests.core__real_data.deep_test_s57_workflow - INFO - 🚀 Starting comprehensive S57 workflow testing...
2025-09-12 21:25:59,946 - tests.core__real_data.deep_test_s57_workflow - INFO - 📊 Phase 1: Testing initial data imports...
2025-09-12 21:25:59,946 - tests.core__real_data.deep_test_s57_workflow - INFO - Testing initial imports across all database formats...
2025-09-12 21:25:59,947 - tests.core__real_data.deep_test_s57_workflow - INFO - Using all 6 S57 files from provided dataset
2025-09-12 21:25:59,948 - tests.core__real_data.deep_test_s57_workflow - INFO - Using 6 ENCs for testing: ['US3CA52M', 'US1EEZ1M', 'US2WC12M', 'US1PO02M', 'US4CA60M', 'US1GC09M']
2025-09-12 21:25:59,948 - tests.core__real_data.deep_test_s57_workflow - INFO - Testing GPKG import...
2025-09-12 21:25:59,949 - src.maritime_module.core.s57_data - INFO - Debug logging enabled. This may impact performance for large datasets.
2025-09-12 21:25:59,956 - src.maritime_module.core.s57_data - INFO 

Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.


2025-09-12 21:26:00,395 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'DSID' with 6 files
2025-09-12 21:26:00,396 - src.maritime_module.core.s57_data - INFO - Processing layer: IsolatedNode
2025-09-12 21:26:00,472 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'IsolatedNode' with 6 files
2025-09-12 21:26:00,473 - src.maritime_module.core.s57_data - INFO - Processing layer: ConnectedNode
2025-09-12 21:26:00,531 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'ConnectedNode' with 6 files
2025-09-12 21:26:00,531 - src.maritime_module.core.s57_data - INFO - Processing layer: Edge
2025-09-12 21:26:00,593 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'Edge' with 6 files
2025-09-12 21:26:00,594 - src.maritime_module.core.s57_data - INFO - Processing layer: Face


Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation options ignored since an existing layer is
         being appended to.
Warning 1: Layer creation opti

2025-09-12 21:26:00,655 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'Face' with 6 files
2025-09-12 21:26:00,656 - src.maritime_module.core.s57_data - INFO - Processing layer: ADMARE


...

2025-09-12 21:26:07,690 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDRGN' with 3 files
2025-09-12 21:26:07,690 - src.maritime_module.core.s57_data - INFO - Processing layer: LNDMRK


Warning 1: The output driver does not seem to natively support StringList type for field LNAM_REFS. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support IntegerList type for field FFPT_RIND. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field CATLND. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field NATQUA. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field NATSUR. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: La

2025-09-12 21:26:07,974 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LNDMRK' with 4 files
2025-09-12 21:26:07,975 - src.maritime_module.core.s57_data - INFO - Processing layer: LIGHTS


Warning 1: The output driver does not seem to natively support StringList type for field LNAM_REFS. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support IntegerList type for field FFPT_RIND. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field CATLMK. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field COLOUR. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: The output driver does not seem to natively support StringList type for field COLPAT. Converting it to String(JSON) instead. -mapFieldType can be used to control field type conversion.
Warning 1: Th

2025-09-12 21:26:08,363 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'LIGHTS' with 5 files
2025-09-12 21:26:08,364 - src.maritime_module.core.s57_data - INFO - Processing layer: MAGVAR
2025-09-12 21:26:08,774 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'MAGVAR' with 6 files
2025-09-12 21:26:08,775 - src.maritime_module.core.s57_data - INFO - Processing layer: MIPARE
2025-09-12 21:26:09,048 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'MIPARE' with 4 files
2025-09-12 21:26:09,049 - src.maritime_module.core.s57_data - INFO - Processing layer: OBSTRN
2025-09-12 21:26:09,397 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'OBSTRN' with 5 files
2025-09-12 21:26:09,398 - src.maritime_module.core.s57_data - INFO - Processing layer: PIPSOL
2025-09-12 21:26:09,619 - src.maritime_module.core.s57_data - INFO - -> Successfully processed layer 'PIPSOL' with 3 files
2025-09-1